63f78014766fd30436c18a79_Hyperspace - navbar logo.png

# Classic Search With Hyperspace
This notebook demonstrates the use of Hyperspace engine for classic search, combining keyword and value matching.
For more info, see the [Hyperspace documentation](https://docs.hyper-space.io/hyperspace-docs/getting-started/overview).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hyper-space-io/QuickStart/blob/master/DataSets/CrimesInChicago/CrimesInChicago_ClassicSearch.ipynb)
## The Dataset - Crimes In Chicago Dataset
From Kaggle:
"This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days. Data is extracted from the Chicago Police Department's CLEAR (Citizen Law Enforcement Analysis and Reporting) system. In order to protect the privacy of crime victims, addresses are shown at the block level only and specific locations are not identified. This data includes unverified reports supplied to the Police Department. The preliminary crime classifications may be changed at a later date based upon additional investigation and there is always the possibility of mechanical or human error. Therefore, the Chicago Police Department does not guarantee (either expressed or implied) the accuracy, completeness, timeliness, or correct sequencing of the information and the information should not be used for comparison purposes over time."

The [dataset](https://www.kaggle.com/datasets/chicago/chicago-crime) can be downloaded from [Hyperspace git](https://github.com/hyper-space-io/QuickStart/blob/main/DataSets/CrimesInChicago/100k-crimes-dataset-processed_data.zip).

## The Dataset Fields
1. **Case Number {'type': 'keyword'}** - The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.
2. **Date {'type': 'date', 'format': 'MM/dd/yyyy hh:mm:ss a'}** - Date when the incident occurred. this is sometimes a best estimate.
3. **Block {'type 'keyword'}** -The partially redacted address where the incident occurred, placing it on the same block as the actual address.
4. **IUCR {'type 'keyword'}** - The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.
5. **Primary Type {'type 'keyword'}** - The primary description of the IUCR code.
6. **Description {'type 'keyword'}** - The secondary description of the IUCR code, a subcategory of the primary description.
7. **Location Description {'type 'keyword'}** - Description of the location where the incident occurred.
8. **Arrest {'type 'boolean'}** - Indicates whether an arrest was made.
9. **Domestic {'type 'boolean'}** - Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act.
10. **Beat {'type 'integer'}** - Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74.
11. **District {'type 'integer'}** - Indicates the police district where the incident occurred. See the districts at https://data.cityofchicago.org/d/fthy-xz3r.
12. **Ward {'type 'integer'}** - The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76.
13. **Community Area {'type 'integer'}** - Indicates the community area where the incident occurred. Chicago has 77 community areas. See the community areas at https://data.cityofchicago.org/d/cauq-8yn6.
14. **FBI Code {'type 'keyword'}** - Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html.
15. **X Coordinate {'type 'integer'}** - The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.
16. **Y Coordinate {'type 'integer'}** - The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.
17. **Year {'type 'integer'}** - Year the incident occurred.
18. **Updated On {'type 'date', 'format 'MM/dd/yyyy hh:mm:ss a'}** - Date and time the record was last updated.
19. **Latitude {'type 'float'}** - The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.
20. **Longitude {'type 'float'}** - The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.
21. **Location {'type 'geo_point', 'struct_type 'list'}** - The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block.


# Setting up the Hyperspace Environment
Setting up the environment and running the query includes the following steps
1. Download and install the client API
2. Connect to a server
3. Create data schema file
4. Create collection
5. Ingest data
6. Run query

## 1. Install the Hyperspace client API
Hyperspace API can be installed directly from git, using the following command:

In [3]:
pip install git+https://github.com/hyper-space-io/hyperspace-py

  Cloning https://github.com/hyper-space-io/hyperspace-py to /tmp/pip-req-build-axxmqr16
  Running command git clone --filter=blob:none --quiet https://github.com/hyper-space-io/hyperspace-py /tmp/pip-req-build-axxmqr16
  Resolved https://github.com/hyper-space-io/hyperspace-py to commit 70d23409dc1b8be4a73845f17f6b8f84a104b4ea
  Preparing metadata (setup.py) ... done
  Created wheel for hyperspace: filename=hyperspace-1.0.0-py3-none-any.whl size=38874 sha256=7d4a0087925d1704fa1f2ea8111b66ae1305f0b0308dbe1f2cedbc68961126a2
  Stored in directory: /tmp/pip-ephem-wheel-cache-d17ut6v3/wheels/c4/96/59/f4b91d653fdbfc819e48a7dacbea1c9f3de59a1bc113aa840d
Successfully built hyperspace


### Download Dataset

In [6]:
from urllib.request import urlretrieve
import os

def download_data(url, file_name):
    """
    url (str): URL of the file to download.
    file_name (str): Local path where the file will be saved.
    """
    # Check if the file already exists and is not empty
    if os.path.exists(file_name) and os.path.getsize(file_name) > 0:
        print(f"The file {file_name} already exists and is not empty.")
    else:
        try:
            # Attempt to download the file from `url` and save it locally under `file_name`
            urlretrieve(url, file_name)
            # Check if the file was downloaded and is not empty
            if os.path.exists(file_name) and os.path.getsize(file_name) > 0:
                print(f"Successfully downloaded {file_name}")
            else:
                print("Download failed or file is empty.")

        except Exception as e:
            print(f"An error occurred: {e}")

import zipfile
def unzip_file(path_to_zip_file):
    directory_to_extract_to = './'
    try:
        with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
            zip_ref.extractall(directory_to_extract_to)
        print(f'Success! Files have been extracted to {directory_to_extract_to}')

    except zipfile.BadZipFile:  # Handle a bad zip file
        print("Error: The file is a bad zip file. Unable to unzip.")
    except FileNotFoundError:  # Handle the file not being found
        print("Error: The file was not found. Please check the path.")
    except Exception as e:  # Handle other exceptions
        print(f"An error occurred: {e}")


In [7]:
data_url = "https://hyperspace-datasets.s3.eu-central-1.amazonaws.com/100k-crimes-dataset-processed_data.zip"

download_data(data_url, "./100k-crimes-dataset-processed_data.zip")
unzip_file("./100k-crimes-dataset-processed_data.zip")



Successfully downloaded ./100k-crimes-dataset-processed_data.zip
Success! Files have been extracted to ./


## 2. Connect to the server

Once the Hyperspace API is installed, the database can be accessed by creating a local instance of the Hyperspace client. This step requires host address, username and password.

In [9]:
import hyperspace
from getpass import getpass

username    = "USERNAME"
host        = "HOST"
password=getpass()


hyperspace_client = hyperspace.HyperspaceClientApi(host=host, username=username, password=password)
print(hyperspace_client)


We check the status before proceeding, also delete collection if exist ...

In [10]:
collections_info = hyperspace_client.collections_info()
if 'CrimesInChicago' in collections_info['collections'] :
    collections_info = hyperspace_client.delete_collection('CrimesInChicago')
    collections_info = hyperspace_client.collections_info()
collections_info



{'collections': {'DocRetrievalEmbedded': {'creation_time': '2024-03-20T10:50:16Z',
   'last_query_time': '2024-03-25T09:27:27Z',
   'size': 39},
  'GeneratedData': {'creation_time': '2024-03-26T05:27:09Z',
   'last_query_time': '2024-03-26T06:28:50Z',
   'size': 100000},
  'all-MiniLM-L6-v2_arXiv': {'creation_time': '2024-03-26T07:42:43Z',
   'last_query_time': '2024-03-26T08:03:16Z',
   'size': 604001}}}

## 3. Create a Data Schema File

Similarly to other search databases, Hyper-Space database requires a configuration file that outlines the data schema. Here, we create a config file that corresponds to the fields of the given dataset.

For vector fields, we also provide the index type to be used, and the metric. . Current options for index include "**brute_force**", "**hnsw**", "**ivf**", and "**bin_ivf**" for binary vectors, and "**IP**" (inner product) as a metric for floating point vectors and "**Hamming**" ([hamming distance](https://en.wikipedia.org/wiki/Hamming_distance)) for binary vectors.
Note that the key 'low_cardinality' enables faster search for low cardinality fields.

In [ ]:
config = {
  "configuration": {
    "ID": {
      "type": "keyword",
      "id": True
    },
    "Case Number": {
      "type": "keyword"
    },
    "Date": {
      "type": "date",
      "format": "MM/dd/yyyy hh:mm:ss a"
    },
    "Block": {
      "type": "keyword"
    },
    "IUCR": {
      "type": "keyword"
    },
    "Primary Type": {
      "type": "keyword"
    },
    "Description": {
      "type": "keyword"
    },
    "Location Description": {
      "type": "keyword"
    },
    "Arrest": {
      "type": "boolean"
    },
    "Domestic": {
      "type": "boolean"
    },
    "Beat": {
      "type": "integer"
    },
    "District": {
      "type": "integer"
    },
    "Ward": {
      "type": "integer"
    },
    "Community Area": {
      "type": "integer"
    },
    "FBI Code": {
      "type": "keyword"
    },
    "X Coordinate": {
      "type": "integer"
    },
    "Y Coordinate": {
      "type": "integer"
    },
    "Year": {
      "type": "integer"
    },
    "Updated On": {
      "type": "date",
      "format": "MM/dd/yyyy hh:mm:ss a"
    },
    "Latitude": {
      "type": "float"
    },
    "Longitude": {
      "type": "float"
    },
    "Location": {
      "type": "geo_point",
      "struct_type": "list"
    }
  }
}


## 4. Create Collection
The Hyerspace engine stores data in Collections, where each collection commonly hosts data of similar context, etc. Each search is then performed within a collection. We create a collection using the command "**create_collection**(schema_filename, collection_name)".

In [12]:
collection_name = 'CrimesInChicago'

if collection_name not in hyperspace_client.collections_info()["collections"]:
  hyperspace_client.create_collection(config, collection_name)

hyperspace_client.collections_info()


{'collections': {'CrimesInChicago': {'creation_time': '2024-03-26T10:43:12Z',
   'size': 0},
  'DocRetrievalEmbedded': {'creation_time': '2024-03-20T10:50:16Z',
   'last_query_time': '2024-03-25T09:27:27Z',
   'size': 39},
  'GeneratedData': {'creation_time': '2024-03-26T05:27:09Z',
   'last_query_time': '2024-03-26T06:28:50Z',
   'size': 100000},
  'all-MiniLM-L6-v2_arXiv': {'creation_time': '2024-03-26T07:42:43Z',
   'last_query_time': '2024-03-26T08:03:16Z',
   'size': 604001}}}

# 5. Ingest data

In the next step we ingest the dataset in batches of 500 documents. This number can be controlled by the user, and in particular, can be increased in order improve ingestion time. We add batches of data using the command **add_batch**(batch, collection_name).

In [16]:
batch

[{'ID': 10224738,
  'Case Number': 'HY411648',
  'Date': 1441449000,
  'Block': '043XX S WOOD ST',
  'IUCR': '0486',
  'Primary Type': 'BATTERY',
  'Description': 'DOMESTIC BATTERY SIMPLE',
  'Location Description': 'RESIDENCE',
  'Arrest': False,
  'Domestic': True,
  'Beat': 924,
  'District': 9,
  'Ward': 12,
  'Community Area': 61,
  'FBI Code': '08B',
  'X Coordinate': 1165074,
  'Y Coordinate': 1875917,
  'Year': 2015,
  'Updated On': 1518270601,
  'Latitude': 41.815117282,
  'Longitude': -87.669999562,
  'Location': [41.815117282, -87.669999562]}]

In [17]:
import numpy as np
import json
dataset_path = '100k-crimes-dataset-processed_data.json'

BATCH_SIZE = 1000

batch = []
with open(dataset_path) as data:
    for i, metadata_row in enumerate(data):
        row = {key: value for key, value in json.loads(metadata_row).items() if key in config["configuration"].keys()}
        row["ID"] = str(row["ID"])
        batch.append(row)

        if i % BATCH_SIZE == 0:
            response = hyperspace_client.add_batch(batch, collection_name)
            batch.clear()
            print(i, response)
hyperspace_client.commit(collection_name)


0 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
1000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
2000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
3000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
4000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
5000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
6000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
7000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
8000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
9000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
10000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
11000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
12000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
13000 {'code': 200, 'message': 'Batch 

{'code': 200, 'message': 'Dataset committed successfully', 'status': 'OK'}

### Check collection size after ingestion

In [ ]:
print(hyperspace_client.collections_info())

{'collections': {'CrimesInChicago': {'creation_time': '2024-01-18T14:09:54Z', 'size': 100000}}}


## 6. Define Logic and Run a Query
We build a search query using Hyper-space. we first select one document from the datset, document "1022", lets take a look in this document


In [27]:
query_doc = hyperspace_client.get_document(collection_name, "10224738")
query_doc

{'ID': '10224738',
 'Case Number': 'HY411648',
 'Date': 1441449000,
 'Block': '043XX S WOOD ST',
 'IUCR': '0486',
 'Primary Type': 'BATTERY',
 'Description': 'DOMESTIC BATTERY SIMPLE',
 'Location Description': 'RESIDENCE',
 'Arrest': False,
 'Domestic': True,
 'Beat': 924,
 'District': 9,
 'Ward': 12,
 'Community Area': 61,
 'FBI Code': '08B',
 'X Coordinate': 1165074,
 'Y Coordinate': 1875917,
 'Year': 2015,
 'Updated On': 1518270601,
 'Latitude': 41.815117282,
 'Longitude': -87.669999562,
 'Location': [41.815117282, -87.669999562]}

We will use a very simple logic, which matchs the description and location, and make sure case number doesn't match so we won't get back the same result.

We use the following logic:


*   document/case will match if it has the same "Description" and not the same "Case Number"
*   document/case will also match if it happended in the same "Block"
*   we boost the score by 5 points if geo "Location" is close (40 something)
*   we reduce the score by 5 points if the "District" is the same and case date if 40 to 100 days before the query case date

Score function is listed next :



In [24]:
def similarity_score(params, doc):
    score = 0.0
    if match('Description') and not match('Case Number'):
        score = rarity_max('Description')
        if geo_dist_match('Location',40):
            score += 5
        if params['District'] == doc['District'] and window_match('Date', 100, 40):
            score -= 5

    return score

hyperspace_client.set_function(similarity_score, collection_name, "similarity_score")



{'code': 200, 'message': 'Function was set successfully', 'status': 'OK'}

### Next, We Submit The Query

In [36]:
results = hyperspace_client.search({'params': query_doc},
                                        size=30,
                                        function_name='similarity_score',
                                        collection_name=collection_name)

candidates = results['candidates']

print(f"Query run time = {results['took_ms']}ms")
print("-"*180)


print("{:<5} {:<100} {:<40} ".format("Rank", "ID",  "Score"))
print("="*180)

for i, result in enumerate(results['similarity']):
    print("{:<5} {:<100}  {:<10}".format(i + 1, result['document_id'], round(result['score'], 2)))


Query run time = 2.28533ms
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Rank  ID                                                                                                   Score                                    
1     10224751                                                                                              8.47      
2     10224758                                                                                              8.47      
3     10224766                                                                                              8.47      
4     10224787                                                                                              8.47      
5     10224791                                                                                              8.47      
6     10224801                                  

We display the top 30 results. Note that results with similar score are ordered arbitrarily, so more complex logic will likely result in better outcome.
lets take a look in the content of top matching document, we can see the match in "Description", no match in "Block" and "District" and match in "Location"

### Score function with some aggregation

we add count aggregation to get the number of cases with "Description" match and separate count for the number of cases with additional geo match.

In [39]:
def similarity_score_aggregations(params, doc):
    score = 0.0

    if match('Description') and not match('Case Number'):
        aggregate_count("count of cases")

        score = rarity_max('Description')
        if geo_dist_match('Location',40):
            aggregate_count("count of cases in nearby districts")
            score += 5
        if params['District'] == doc['District'] and window_match('Date', 100, 40):
            score -= 5
    if match('Block'):
        score += rarity_max('Block')
    return score

hyperspace_client.set_function(similarity_score_aggregations, collection_name, "similarity_score_aggregations")



{'code': 200, 'message': 'Function was set successfully', 'status': 'OK'}

### Next, we fire the query

In [49]:
results = hyperspace_client.search({'params': query_doc},
                                        size=30,
                                        function_name='similarity_score_aggregations',
                                        collection_name=collection_name)


candidates = results['candidates']

print(f"Query run time = {results['took_ms']}ms")
print("-"*180)


print("{:<5} {:<100} {:<40} ".format("Rank", "ID",  "Score"))
print("="*180)

for i, result in enumerate(results['similarity']):
    print("{:<5} {:<100}  {:<10}".format(i + 1, result['document_id'], round(result['score'], 2)))
print("="*180)

print("Count of initial candidates ", results['aggregations']["count of cases"]['count'])
print("Count of cases in nearby districts ", results['aggregations']["count of cases in nearby districts"]['count'])

Query run time = 2.43376ms
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Rank  ID                                                                                                   Score                                    
1     10192664                                                                                              21.98     
2     10325639                                                                                              21.98     
3     10224738                                                                                              13.51     
4     10263422                                                                                              13.51     
5     10269178                                                                                              13.51     
6     10318418                                  

we can see the aggregation section in the results where the two counts are listed



For more information, visit us at [Hyperspace](https://www.hyper-space.io/)